# Linear Modeling of House Price

## 1. Introduction

## 2. Preparing the Dataset

In [341]:
import pandas as pd
import numpy as np
from Modules.dataexploration import DataExploration

pd.set_option('display.float', '{:,.2f}'.format)
pd.set_option('display.max_columns', 75)
pd.set_option('display.max_rows', 75)
pd.set_option('display.max_info_columns', 50)

In [342]:
house_df = pd.read_csv('Dataset\jabodetabek_house_data_prcsd.csv')

### 2.1. Subsetting the Test Set for Assessing Prediction

In [343]:
from sklearn.model_selection import train_test_split

X = house_df.drop('price_in_rp', axis=1)
y = house_df.price_in_rp

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=99
)

### 2.2. Missing Values

First let's review missing data from our original dataset (yes, not `train` or `test set`). By evaluating `original dataset`, we analyze the possibility of missing value for both scenarios.

In [344]:
numerical_features = house_df.select_dtypes(exclude=['object', 'category']).columns
DataExploration(house_df).show_nans_or_zeroes('nans', numerical_features)

,Nans Count,Nans Percentage (%),Data Types
price_in_rp,0,0.00,float64
lat,0,0.00,float64
long,0,0.00,float64
bedrooms,33,0.93,float64
bathrooms,29,0.82,float64
land_size_m2,2,0.06,float64
building_size_m2,2,0.06,float64
carports,0,0.00,int64
maid_bedrooms,0,0.00,int64
maid_bathrooms,0,0.00,int64


In [345]:
non_numeric_features = house_df.select_dtypes(include=['object']).columns
DataExploration(house_df).show_nans_or_zeroes('nans', non_numeric_features)

,Nans Count,Nans Percentage (%),Data Types
url,0,0.00,object
title,0,0.00,object
address,0,0.00,object
district,0,0.00,object
city,0,0.00,object
facilities,0,0.00,object
property_type,1,0.03,object
ads_id,4,0.11,object
certificate,141,3.99,object
electricity,0,0.00,object


Remember that those observations (from our original dataset) with missing records may exist in `training set`, `test set` or may be in both. Therefore, we must develop method to handle missing value that works for `training set`, `test set`, and even `potential future data` which we want to predict if any. Possible method:

1. Drop observations that have missing `bathrooms` or `bedrooms`, `land_size` and `building_size` records
2. Drop `building_age`, `year_built`, and `building_orientation` which have a lot of missing value
3. For missing `floors` and `electricity_num`, will be imputed. Features that used for imputation is best choosen according to its correlation coefficient (previous chapter).
4. For missing `certificate`, let's assume `certificate = lainnya`.
5. For missing `property_condition` and `furnishing`, let's assume majority of data.

According to above explanation, it should be noted that if there are any `future data` that we want to predict, the only missing features that can be considered are `floors`, `electricity`, `certificate`, `property_condition` and `furnishing`. Other than that, the prediction can't be performed.

In below codes, note that the imputation is fitted only with `train set` to ensure that the prediction model is isolated from `test set` (avoding data leakage)

In [346]:
from sklearn.impute import SimpleImputer, KNNImputer

# 1. Drop observations that missing vital records
X_train = X_train\
    .dropna(subset=['bedrooms', 'bathrooms', 'building_size_m2', 'land_size_m2'],)\
    .drop(['building_age', 'year_built', 'building_orientation'], axis=1)\
    .reset_index(drop=True) # 2. Drop features with lot of NaN
# 3.a. Impute missing `floors`
knn_imputer = KNNImputer(n_neighbors=1)
missing_floors_index = X_train[X_train.floors.isna()].index
features_estimator_floors = ['building_size_m2', 'bedrooms', 'bathrooms', 'floors']
imputer_floors = knn_imputer.fit(X_train[features_estimator_floors])
imputed_data_floors = imputer_floors.transform(X_train[features_estimator_floors])
X_train = X_train.assign(
    floors = imputed_data_floors[:, 3]
)
# 3.b. Impute missing `Electricity`
features_estimator_electricity = [
    'building_size_m2', 'bedrooms', 'bathrooms', 'floors', 'electricity_num'
]
missing_electricity_index = X_train[X_train.electricity_num.isna()].index
imputer_electricity = knn_imputer.fit(X_train[features_estimator_electricity])
imputed_data_electricity = imputer_electricity.transform(X_train[features_estimator_electricity])
X_train = X_train.assign(
    electricity_num = imputed_data_electricity[:, 4]
)
# 4. Impute missing `certificate`
constant_imputer = SimpleImputer(strategy='constant', fill_value='lainnya (ppjb,girik,adat,dll)')
X_train['certificate'] = constant_imputer.fit_transform(X_train[['certificate']]).flatten()
# 5. Impute missing `property_condition` and `furnishing`
frequent_imputer = SimpleImputer(strategy='most_frequent')
X_train['property_condition'] = frequent_imputer\
    .fit_transform(X_train[['property_condition']])\
    .flatten()
X_train['furnishing'] = frequent_imputer\
    .fit_transform(X_train[['furnishing']])\
    .flatten()

Let's see the imputation result:

In [347]:
# missing floor imputation results
show_features = ['building_size_m2', 'land_size_m2', 'bedrooms', 'bathrooms', 'floors']
X_train.iloc[missing_floors_index, X_train.columns.get_indexer(show_features)]

,building_size_m2,land_size_m2,bedrooms,bathrooms,floors
474,358.00,200.00,4.00,4.00,3.00
2262,200.00,187.00,3.00,3.00,2.00
2766,250.00,320.00,4.00,4.00,2.00


In [348]:
# missing electricity imputation results
show_features = ['building_size_m2', 'land_size_m2', 'bedrooms', 'bathrooms', 'floors', 'electricity_num']
X_train.iloc[missing_electricity_index, X_train.columns.get_indexer(show_features)].sample(5, random_state=101)

,building_size_m2,land_size_m2,bedrooms,bathrooms,floors,electricity_num
2298,255.00,225.00,1.00,1.00,1.00,"1,300.00"
914,110.00,78.00,3.00,3.00,2.00,"2,200.00"
2841,"1,800.00",890.00,54.00,54.00,5.00,"53,000.00"
2662,650.00,763.00,4.00,4.00,2.00,"16,500.00"
1627,500.00,390.00,4.00,4.00,2.00,"3,300.00"


### 2.3. Unnecessary Features and Features with High Cardinality

In [349]:
cols_to_drop = ['url', 'title', 'address', 'district', 'ads_id', 'facilities', 'property_type', 'electricity']
X_train.drop(cols_to_drop, axis=1, inplace=True)

### 2.4. Conclusion: Prepared Features

In [350]:
numerical_features = X_train.select_dtypes(exclude=['object', 'category']).columns
DataExploration(X_train).show_nans_or_zeroes('nans', numerical_features)

,Nans Count,Nans Percentage (%),Data Types
lat,0,0.00,float64
long,0,0.00,float64
bedrooms,0,0.00,float64
bathrooms,0,0.00,float64
land_size_m2,0,0.00,float64
building_size_m2,0,0.00,float64
carports,0,0.00,int64
maid_bedrooms,0,0.00,int64
maid_bathrooms,0,0.00,int64
floors,0,0.00,float64


In [351]:
non_numeric_features = X_train.select_dtypes(include=['object']).columns
DataExploration(X_train).show_nans_or_zeroes('nans', non_numeric_features)

,Nans Count,Nans Percentage (%),Data Types
city,0,0.00,object
certificate,0,0.00,object
property_condition,0,0.00,object
furnishing,0,0.00,object


### 2.4. Additional: Pipeline for Test Set

#### **Features-Dropper Transformer**

In [352]:
from sklearn.base import BaseEstimator, TransformerMixin

class FeaturesDropper(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.cols_dropped = []

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        drop_list = [
            'building_age', 'year_built', 'building_orientation',
            'url', 'title', 'address', 'district', 'ads_id', 'facilities', 
            'property_type', 'electricity'
            ]
        # checking if streamed data has unused features
        self.cols_dropped +=\
            [column for column in X.columns if column in drop_list]
        if self.cols_dropped:
            X_new = X.drop(self.cols_dropped, axis=1)
        return X_new

#### **Transformer for Specific Columns**

In [353]:
class FloorsKNNImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self._knn_imputer = KNNImputer(n_neighbors=1)
        self._features_estimator_floors = ['building_size_m2', 'bedrooms', 'bathrooms', 'floors']

    def fit(self, X, y=None):
        X_fit = X.copy()
        self._knn_imputer = self._knn_imputer.fit(X_fit[self._features_estimator_floors])
        return self

    def transform(self, X, y=None):
        X_transf = X.copy()
        imputed_data_floors = self._knn_imputer.transform(X_transf[self._features_estimator_floors])
        X_new = X_transf.assign(
            floors = imputed_data_floors[:, 3]
        )
        return X_new

class ElectricityKNNImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self._knn_imputer = KNNImputer(n_neighbors=1)
        self._features_estimator_electricity = \
            ['building_size_m2', 'bedrooms', 'bathrooms', 'floors', 'electricity_num']

    def fit(self, X, y=None):
        X_fit = X.copy()
        self._knn_imputer = self._knn_imputer.fit(X_fit[self._features_estimator_electricity])
        return self

    def transform(self, X, y=None):
        X_transf = X.copy()
        imputed_data_electricity = self._knn_imputer.transform(X_transf[self._features_estimator_electricity])
        X_new = X_transf.assign(
            electricity_num = imputed_data_electricity[:, 4]
        )
        return X_new

#### **Creating Pipeline for Train-Test Set**

In [354]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn import set_config

features_dropper = FeaturesDropper()
floors_imputer = FloorsKNNImputer()
electricity_imputer = ElectricityKNNImputer()
certificate_imputer = SimpleImputer(strategy='constant', fill_value='lainnya (ppjb,girik,adat,dll)')
features_transf = ColumnTransformer([
    ('certificate_impute', certificate_imputer, ['certificate']),
    ('property_condition_impute', SimpleImputer(strategy='most_frequent'), ['property_condition']),
    ('furnishing_impute', SimpleImputer(strategy='most_frequent'), ['furnishing'])
    ], remainder='passthrough')
dataset_transform_pipeline = make_pipeline(
    features_dropper,
    floors_imputer,
    electricity_imputer,
    features_transf
    )

set_config(display='diagram')
dataset_transform_pipeline

Pipeline(steps=[('featuresdropper', FeaturesDropper()),
                ('floorsknnimputer', FloorsKNNImputer()),
                ('electricityknnimputer', ElectricityKNNImputer()),
                ('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('certificate_impute',
                                                  SimpleImputer(fill_value='lainnya '
                                                                           '(ppjb,girik,adat,dll)',
                                                                strategy='constant'),
                                                  ['certificate']),
                                                 ('property_condition_impute',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['property_condition']),
                                                 ('furnishing_impute',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['furnishing'])]))])

In [355]:
X = house_df.drop('price_in_rp', axis=1)
y = house_df.price_in_rp
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=99
)
X_train = X_train\
    .dropna(subset=['bedrooms', 'bathrooms', 'building_size_m2', 'land_size_m2'])\
    .reset_index()
columns_to_drop = [
            'building_age', 'year_built', 'building_orientation',
            'url', 'title', 'address', 'district', 'ads_id', 'facilities', 
            'property_type', 'electricity'
            ]
dataset_columns = X_train.drop(columns_to_drop, axis=1).columns
pipeline = dataset_transform_pipeline.fit(X_train)
X_train = pd.DataFrame(pipeline.transform(X_train), columns=dataset_columns)

In [356]:
DataExploration(X_train).show_nans_or_zeroes('nans', numerical_features)

,Nans Count,Nans Percentage (%),Data Types
lat,0,0.00,object
long,0,0.00,object
bedrooms,0,0.00,object
bathrooms,0,0.00,object
land_size_m2,0,0.00,object
building_size_m2,0,0.00,object
carports,0,0.00,object
maid_bedrooms,0,0.00,object
maid_bathrooms,0,0.00,object
floors,0,0.00,object


In [357]:
DataExploration(X_train).show_nans_or_zeroes('nans', non_numeric_features)

,Nans Count,Nans Percentage (%),Data Types
city,0,0.00,object
certificate,0,0.00,object
property_condition,0,0.00,object
furnishing,0,0.00,object


In [363]:
# missing floor imputation results
X_train['lat']

0          unfurnished
1       semi furnished
2          unfurnished
3          unfurnished
4          unfurnished
             ...      
3141       unfurnished
3142       unfurnished
3143       unfurnished
3144       unfurnished
3145       unfurnished
Name: lat, Length: 3146, dtype: object

## 3. Linear Modeling: Ordinary Least Square